# Chapter 1. Elementary Matrix Algebra

This chapter covers the following topics:

- [1.1 Matrix Norms](?kernel_name=python3#1.1-Matrix-Norms)
- [1.2 Matrix Decomposition](?kernel_name=python3#1.2-Matrix-Decompositions)
 - [QR Decomposition](?kernel_name=python3#QR-Decomposition)
 - [Singular Value Decomposition (SVD)](?kernel_name=python3#Singular-Value-Decompostion)
 - [The Truncated SVD](?kernel_name=python3#The-Rank-k-Truncated-SVD)
 - [Eigenvalue Decomposition](?kernel_name=python3#Eigenvalue-Decompostion)
- [1.3 Matrix Inverse and Pseudo-inverse](?kernel_name=python3#1.3-Matrix-Inverse-and-Pseudo-inverse)
 - [Matrix Inverse](?kernel_name=python3#Matrix-Inverse)
 - [Matrix Pseudo-inverse](?kernel_name=python3#Matrix-Pseudo-inverse)
 - [Least Squares Regression (LSR)](?kernel_name=python3#Least-Squares-Regression)
 
You can skip this chapter if you are already with elementary matrix operations and NumPy.


Let's first generate a $7\times 4$ matrix ${\bf A}$, whose entries are i.i.d. Gaussians.

In [1]:
import numpy as np

np.random.seed(123)
matrixA = np.random.standard_normal([7, 4])
print("A = \n\n", matrixA)

A = 

 [[-1.0856306   0.99734545  0.2829785  -1.50629471]
 [-0.57860025  1.65143654 -2.42667924 -0.42891263]
 [ 1.26593626 -0.8667404  -0.67888615 -0.09470897]
 [ 1.49138963 -0.638902   -0.44398196 -0.43435128]
 [ 2.20593008  2.18678609  1.0040539   0.3861864 ]
 [ 0.73736858  1.49073203 -0.93583387  1.17582904]
 [-1.25388067 -0.6377515   0.9071052  -1.4286807 ]]


## 1.1 Matrix Norms

The matrix Frobenius norm is defined by
$$ \| {\bf A} \|_F = \sqrt{\sum_{i j} a_{i j}^2} .$$

The matrix spectral norm is defined by
$$ \| {\bf A} \|_2 \; = \; \max_{{\bf x} \neq {\bf 0}} \frac{\|{\bf A} {\bf x}\|_2}{ \| {\bf x}\|_2} .$$

Let's see the Frobenius norm and spectral norm of ${\bf A}$.

In [135]:
frobeniusNormA = np.linalg.norm(matrixA, 'fro')
print("The Frobenius norm of A is:", frobeniusNormA)

spectralNormA = np.linalg.norm(matrixA, 2)
print("The spectral norm of A is:", spectralNormA)

The Frobenius norm of A is: 6.33811103512
The spectral norm of A is: 4.17282826794


## 1.2 Matrix Decompositions

### QR Decomposition

Let ${\bf A}$ be an $m\times n$ matrix with $m \geq n$.
The QR decomposition of ${\bf A}$ is
$$
{\bf A} \; = \; \underbrace{{\bf Q_A}}_{m\times n} \: \underbrace{{\bf R_A}}_{n\times n}.
$$
The matrix ${\bf Q_A}$ has orthonormal columns, that is,
${{\bf Q}_{\bf A}^T} {\bf Q_A} = {\bf I}_n$.
The matrix ${\bf R_A}$ is upper triangular, that is, for all $i < j$, the $(i,j)$-th entry of ${\bf R_A}$ is zero.
It costs $O (m n^2)$ time to compute the QR decompostion.

Let's compute the QR decomposition by NumPy ([numpy.linalg.qr](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.qr.html#numpy.linalg.qr)).

In [136]:
matrixQ, matrixR = np.linalg.qr(matrixA, mode='reduced')
print("Q = \n\n", matrixQ, "\n\n")
print("R = \n\n", matrixR, "\n\n")

Q = 

 [[-0.30926928 -0.35898426  0.1875401  -0.53139597]
 [-0.16482889 -0.5183199  -0.68687346 -0.25868406]
 [ 0.3606339   0.33210471 -0.31425974 -0.29212158]
 [ 0.42485998  0.27965247 -0.22101851 -0.48596311]
 [ 0.62841486 -0.50081084  0.47145063 -0.18458948]
 [ 0.21005805 -0.38924632 -0.21486539  0.3611136 ]
 [-0.35719956  0.10794234  0.27747498 -0.40586871]] 


R = 

 [[ 3.51030859  0.75048109 -0.01062386  1.31785587]
 [ 0.         -3.42479135  0.76593325 -0.19517926]
 [ 0.          0.          2.95750534 -0.32911936]
 [ 0.          0.          0.          2.08331573]] 




In [137]:
# Verify the result by multiplying Q and R
print("Round-off error:\n")
print("\t ||A - Q R|| = ", np.linalg.norm(matrixA - np.dot(matrixQ, matrixR)))

Round-off error:

	 ||A - Q R|| =  1.15910686703e-15


Let's check if the columns of ${\bf Q}$ are orthonormal.If so, we should have $\|{\bf I}_n - {\bf Q}^T {\bf Q}\|_F = 0$. (However, due to the round-off error, it cannot be exactly zero.)

In [138]:
errorTerm = np.eye(matrixQ.shape[1]) - np.dot(matrixQ.transpose(), matrixQ)
print("Round-off error:\n")
print("\t ||I - Q' Q|| = ", np.linalg.norm(errorTerm, 'fro'))

Round-off error:

	 ||I - Q' Q|| =  5.64934213789e-16


### Singular Value Decompostion

Let ${\bf A}$ be an $m\times n$ matrix and $r = \min \{m, n\}$.
The singular value decomposition (SVD) of ${\bf A}$ is
$$
\underbrace{{\bf A} }_{m\times n}
\; = \; \underbrace{{\bf U}_{\bf A}}_{m\times r} 
    \: \underbrace{{\bf \Sigma_A}}_{r \times r} 
    \: \underbrace{{\bf V}_{\bf A}^T}_{r\times n}
\; = \; \sum_{i = 1}^{\rho} \sigma_{{\bf A},i} {\bf u}_{{\bf A},i} {\bf v}_{{\bf A} , i}^T.
$$
Here $\sigma_{{\bf A},1} \geq \cdots \geq \sigma_{{\bf A},r} > 0$ are the singular values,
${\bf u}_{{\bf A},1} , \cdots , {\bf u}_{{\bf A},r} \in \mathbb{R}^m$ are the left singular vectors,
and ${\bf v}_{{\bf A},1} , \cdots , {\bf v}_{{\bf A},r} \in \mathbb{R}^n$ are the right singular vectors.

Let's compute the SVD of ${\bf A} = {\bf U_A} {\bf \Sigma_A} {\bf V}_{\bf A}^T$ using NumPy ([numpy.linalg.svd](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html#numpy.linalg.svd)). The SVD costs $O (m n r)$ time, where $r = \min \{m, n\}$.

In [93]:
# SVD
matrixU, vectorSigma, matrixV = np.linalg.svd(matrixA, full_matrices=False)
print("U = \n\n", matrixU, "\n\n")
print("sigma = \n\n", vectorSigma, "\n\n")
print("V = \n\n", matrixU, "\n\n")

U = 

 [[-0.1863171   0.40039923 -0.45006716 -0.38010012]
 [ 0.2494457   0.79259629  0.27603966 -0.26058921]
 [ 0.10899075 -0.24297464  0.40506454 -0.43526038]
 [ 0.13150461 -0.26082502  0.27088106 -0.62050071]
 [ 0.63539313 -0.20851669 -0.63220244 -0.23488751]
 [ 0.49373833  0.19477821  0.07198984  0.29207641]
 [-0.47580287  0.05465757 -0.28093659 -0.26956198]] 


sigma = 

 [ 4.17282827  3.39075521  2.77573984  1.88605493] 


V = 

 [[-0.1863171   0.40039923 -0.45006716 -0.38010012]
 [ 0.2494457   0.79259629  0.27603966 -0.26058921]
 [ 0.10899075 -0.24297464  0.40506454 -0.43526038]
 [ 0.13150461 -0.26082502  0.27088106 -0.62050071]
 [ 0.63539313 -0.20851669 -0.63220244 -0.23488751]
 [ 0.49373833  0.19477821  0.07198984  0.29207641]
 [-0.47580287  0.05465757 -0.28093659 -0.26956198]] 




In [94]:
# Verify the result by multiplying U, Sigma, V'
matrixUSV = np.dot(matrixU * vectorSigma, matrixV)
print("Round-off error:\n")
print("\t ||A - U diag(sigma) V'|| = ", np.linalg.norm(matrixA - matrixUSV))

Round-off error:

	 ||A - U diag(sigma) V'|| =  5.99700303564e-15


The columns of ${\bf U}_{\bf A}$ are the vectors ${\bf u}_{{\bf A},1} , \cdots , {\bf u}_{{\bf A},\rho} \in \mathbb{R}^m$. The columns of ${\bf V}_{\bf A}$ are the vectors ${\bf v}_{{\bf A},1} , \cdots , {\bf v}_{{\bf A},\rho} \in \mathbb{R}^m$.

The matrices ${\bf U}_{\bf A}$ and ${\bf V}_{\bf A}$ have orthonormal columns. The readers are encouraged to verify that ${\bf U}_{\bf A}^T {\bf U}_{\bf A}= {\bf I}_\rho$
and ${\bf V}_{\bf A}^T {\bf V}_{\bf A}= {\bf I}_\rho$.

### The Rank k Truncated SVD

In applications such as the principal component analysis (PCA), latent semantic indexing (LSI), word2vec, spectral clustering,
we are only interested in the top $k$ ($\ll m, n$) singular values and singular vectors.
The rank $k$ truncated SVD ($k$SVD) is denoted by
$$
{\bf A}_k
\; = \;
\sum_{i = 1}^{k} \sigma_{{\bf A},i} {\bf u}_{{\bf A},i} {\bf v}_{{\bf A} , i}^T
\; = \; \underbrace{{\bf U}_{{\bf A},k}}_{m\times k} \underbrace{{\bf \Sigma}_{{\bf A},k}}_{k\times k} \underbrace{{\bf V}_{{\bf A},k}^T}_{k\times n} .
$$
Here ${\bf U}_{{\bf A}, k}$ consists of the first $k$ singular vectors of ${\bf U}_{\bf A}$, and ${\bf \Sigma}_{{\bf A},k}$ and ${\bf V}_{{\bf A},k}$ are analogously defined.
Among all the $m\times n$ rank $k$ matrices,
${\bf A}_k$ is the closest approximation to ${\bf A}$ in that
$$
{\bf A}_k \; = \;
\mathrm{argmin}_{{\bf X}} \|{\bf A} - {\bf X} \|^2 ,
\qquad
\mathrm{s.t.} \; \mathrm{rank} ({\bf X}) \leq k.
$$
Here $\|\cdot\|$ denote all unitarily invariant norms, including the Frobenius norm and the spectral norm.

### Eigenvalue Decompostion

The eigenvalue decomposition of an $n\times n$ symmetric matrix ${\bf A}$ is defined by
$$
{\bf A}
\; = \; {\bf U}_{\bf A} {\bf \Lambda}_{\bf A} {\bf U}_{\bf A}^T
\; = \; \sum_{i=1}^n \lambda_{{\bf A},i} {\bf u}_{{\bf A} , i} {\bf u}_{{\bf A} , i}^T .
$$
Here $\lambda_{{\bf A},1} \geq \cdots \geq \lambda_{{\bf A},n}$ are the eigenvalues of ${\bf A}$,
and ${\bf u}_{{\bf A},1} , \cdots , {\bf u}_{{\bf A},n} \in \mathbb{R}^n$ are the corresponding eigenvectors.
${\bf A}$ is called symmetric positive semidefinite (SPSD) if and only if all the eigenvalues are greater than or equal to zero.
If ${\bf A}$ is SPSD,
the SVD and eigenvalue decomposition of ${\bf A}$ are identical.


Let's compute the eigenvalue decomposition of symmetric matrix using NumPy ([numpy.linalg.eigh](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eigh.html#numpy.linalg.eigh)). To begin with, we need to generate a symmetric matrix.

In [95]:
# generate a 4-by-4 random matrix
np.random.seed(123)
matrixAsymmetric = np.random.standard_normal([4, 4])
# make it symmetric
matrixSymmetric = matrixAsymmetric + matrixAsymmetric.transpose()
print("A = \n\n", matrixSymmetric)

A = 

 [[-2.17126121  0.41874519  1.54891476 -0.01490509]
 [ 0.41874519  3.30287307 -3.29341965 -1.06781463]
 [ 1.54891476 -3.29341965 -1.3577723  -0.53869093]
 [-0.01490509 -1.06781463 -0.53869093 -0.86870255]]


By performing the eigenvalue decomposition, we will obtain a vector of eigenvalues and a matrix of eigenvectors. But unlike numpy.linalg.svd, the resulting eigenvalues may not be sorted.

In [96]:
vectorLambda, matrixU = np.linalg.eigh(matrixSymmetric)
print("lambda = \n\n", vectorLambda, "\n\n")
print("U = \n\n", matrixU)

lambda = 

 [-4.4164689  -1.51083014 -0.27273924  5.10517529] 


U = 

 [[ 0.55857377 -0.70567163 -0.43359026  0.04497083]
 [-0.36442388 -0.24157492 -0.16794268 -0.88353384]
 [-0.71322675 -0.24155065 -0.47888806  0.45125061]
 [-0.21563436 -0.62074342  0.74462136  0.11712584]]


Let's check the result by computing $\| {\bf A} - {\bf U} {\bf \Lambda} {\bf U}^T\|_F$.
The result should be very close to zero.

In [100]:
# Verify the result by multiplying U, Lambda, U'
matrixULU = np.dot(matrixU * vectorLambda, matrixU.transpose())
print("Round-off error:\n")
print("\t ||A - U * diag(lambda) * U'|| = ", np.linalg.norm(matrixSymmetric - matrixULU))

Round-off error:

	 ||A - U * diag(lambda) * U'|| =  3.15411656618e-15


## 1.3 Matrix Inverse and Pseudo-inverse

### Matrix Inverse
For an $n\times n$ square matrix ${\bf A}$,
the matrix inverse exists if ${\bf A}$ is non-singular ($\mathrm{rank} ({\bf A}) = n$).
Let ${\bf A}^{-1}$ be the inverse of ${\bf A}$.
Then ${\bf A} {\bf A}^{-1} = {\bf A}^{-1} {\bf A} = {\bf I}_n$.

Let's compute the matrix inversion by ([numpy.linalg.inv](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.inv.html#numpy.linalg.inv)).
We first generate a square matrix (not necessarily symmetric) and then computes its inverse.

In [97]:
# generate a 4-by-4 random matrix
np.random.seed(123)
matrixAsymmetric = np.random.standard_normal([4, 4])
matrixInverse = np.linalg.inv(matrixAsymmetric)

Now let's find out what is ${\bf A}^{-1} {\bf A}$ and ${\bf A} {\bf A}^{-1}$.
Hopefully, both of them should be the identity matrix.

In [101]:
print("inv(A) * A = \n\n", np.dot(matrixInverse, matrixAsymmetric), "\n\n")
print("A * inv(A) = \n\n", np.dot(matrixAsymmetric, matrixInverse), "\n\n")

inv(A) * A = 

 [[  1.00000000e+00   2.22044605e-16   0.00000000e+00   2.22044605e-16]
 [ -4.44089210e-16   1.00000000e+00   1.11022302e-16   2.22044605e-16]
 [ -2.22044605e-16   1.11022302e-16   1.00000000e+00   1.11022302e-16]
 [ -1.38777878e-16   9.71445147e-17  -2.77555756e-17   1.00000000e+00]] 


A * inv(A) = 

 [[  1.00000000e+00  -8.32667268e-17   4.44089210e-16  -3.05311332e-16]
 [  5.55111512e-17   1.00000000e+00   2.77555756e-16  -3.60822483e-16]
 [  2.77555756e-17  -1.21430643e-17   1.00000000e+00  -8.32667268e-17]
 [  0.00000000e+00  -6.93889390e-18   2.77555756e-16   1.00000000e+00]] 




### Matrix Pseudo-inverse


Only square and nonsingular (aka.full rank) matrices have inverse.
For the general rectangular matrices or rank deficient matrices,
matrix pseudo-inverse is used as a generalization of matrix inverse.

Let ${\bf A}$ be any $m\times n$ matrix and $\rho = \mathrm{rank}({\bf A}) \leq m, n$.
Let 
$$
{\bf A} \; =\; 
\underbrace{{\bf U_A}}_{m\times \rho} 
\:\underbrace{{\bf \Sigma_A}}_{\rho\times \rho} 
\:\underbrace{{\bf V}_{\bf A}^T}_{\rho\times n}
$$ 
be the condensed SVD, which means that only the non-zero singular values are retained.
The Moore-Penrose inverse is the most widely used pseudo-inverse, which is defined by
$$
{\bf A}^{\dagger} \; = \;
{\bf V}_{{\bf A}} {\bf \Sigma}_{\bf A}^{-1} {\bf U}_{{\bf A}}^T 
\; \in \; \mathbb{R}^{n\times m}.
$$
Suppose that $m \geq n$. Then it holds that
$$
 {\bf A}^{\dagger} {\bf A} = {\bf I}_n .
$$
However, in general
$$
 {\bf A} {\bf A}^{\dagger} \neq {\bf I}_m .
$$
The matrix ${\bf A} {\bf A}^{\dagger} $ is called the orthogonal projector because ${\bf A} {\bf A}^{\dagger} {\bf B}$ projects ${\bf B}$ to the column space of ${\bf A}$.

Let's compute the matrix pseudo-inverse by the NumPy function [numpy.linalg.pinv](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.pinv.html#numpy.linalg.pinv).

In [102]:
# Generate a 4-by-3 matrix
np.random.seed(123)
matrixA = np.random.standard_normal([4, 3])
matrixPinv = np.linalg.pinv(matrixA)

Now let's find out what is ${\bf A}^{\dagger} {\bf A}$ and ${\bf A} {\bf A}^{\dagger}$.
Only the former should be identity.

In [103]:
print("pinv(A) * A = \n\n", np.dot(matrixPinv, matrixA), "\n\n")
print("A * pinv(A) = \n\n", np.dot(matrixA, matrixPinv), "\n\n")

pinv(A) * A = 

 [[  1.00000000e+00   4.99600361e-16  -3.40005801e-16]
 [  2.77555756e-16   1.00000000e+00  -2.15105711e-16]
 [  0.00000000e+00   2.77555756e-16   1.00000000e+00]] 


A * pinv(A) = 

 [[ 0.87576168 -0.15866457  0.22077876 -0.18677647]
 [-0.15866457  0.7973697   0.28195624 -0.23853195]
 [ 0.22077876  0.28195624  0.60766322  0.33191272]
 [-0.18677647 -0.23853195  0.33191272  0.7192054 ]] 




### Least Squares Regression

Suppose we are given $n$ data points of $d$ dimension, each datum is associated with a vector of ``response''.
Let each row of ${\bf X} \in \mathbb{R}^{n\times d}$ be a datum and each row of ${\bf Y} \in \mathbb{R}^{n\times b}$ be the associated response.Assume that $n \geq d, b$. We hope to find a matrix ${\bf W} \in \mathbb{R}^{d\times b}$ such that ${\bf X} {\bf W} \approx {\bf Y}$. The most straightforward approach is to solve the least squares regression (LSR) problem:
$$
{\bf W}^\star \; = \;
\| {\bf X} {\bf W} - {\bf Y} \|_F^2 .
$$
This problem has closed form solution:
$$
{\bf W}^\star \; = \;{\bf X}^{\dagger}  {\bf Y} .
$$
Let's check it using NumPy. We generate a $7\times 4$ data matrix ${\bf X}$ and a $4\times 2$ model matrix ${\bf W}$. 
The response matrix is generated by
$$
{\bf Y}
\; = \; {\bf X} {\bf W} + \mathrm{Noise} .
$$
We will check if the solution ${\bf W}^\star={\bf X}^{\dagger}  {\bf Y}$ is close to the ground truth ${\bf W}$.

In [133]:
np.random.seed(123)
# Generate a 7-by-4 matrix X
matrixX = np.random.standard_normal([7, 4])
# Generate a 4-by-2 matrix W
matrixWTrue = np.random.standard_normal([4, 2])
# Compute Y by Y = X * W + Noise
noiseRate = 0.04
matrixY = np.dot(matrixX, matrixWTrue) + noiseRate * np.random.standard_normal([7, 2])
print("W (true) = \n\n", matrixWTrue, "\n\n")

# Solve the Least Squares Regression problem: min_W ||X*W - Y||
matrixXPinv = np.linalg.pinv(matrixX)
matrixWSolved = np.dot(matrixXPinv, matrixY)
print("W (solved by pseudo-inverse) = \n\n", matrixWSolved)

W (true) = 

 [[-0.14006872 -0.8617549 ]
 [-0.25561937 -2.79858911]
 [-1.7715331  -0.69987723]
 [ 0.92746243 -0.17363568]] 


W (solved by pseudo-inverse) = 

 [[-0.14973502 -0.8727113 ]
 [-0.24295372 -2.78692194]
 [-1.75762877 -0.69008784]
 [ 0.94772657 -0.19226652]]


We can see that the solution ${\bf W}^\star$ is pretty close to the ground truth ${\bf W}$.
In fact, as ``noiseRate'' goes to zero, the solution should be exact.

In the above, we use the matrix pseudo-inverse to find the solution. However, this method is slow and instable. In practice people usually solve LSR by the conjugate gradient algorithm. It is implemented as the NumPy function [numpy.linalg.lstsq](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html#numpy.linalg.lstsq).

We show the result of [numpy.linalg.lstsq](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html#numpy.linalg.lstsq) in the blow. We can see that the result is the same.

In [132]:
# Solve the Least Squares Regression problem: min_W ||X*W - Y||
matrixWSolved = np.linalg.lstsq(matrixX, matrixY)[0]
print("W (solved by lstsq) = \n\n", matrixWSolved)

W (solved by lstsq) = 

 [[-0.14973502 -0.8727113 ]
 [-0.24295372 -2.78692194]
 [-1.75762877 -0.69008784]
 [ 0.94772657 -0.19226652]]
